In [13]:
from rdkit import Chem
from rdkit.Chem import QED
import pandas as pd
from tqdm import tqdm
import time
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
# now you can import sascore!
import sascorer
import numpy as np
from rdkit.Chem import MACCSkeys

In [14]:
# fname = './eval_gtp2_20002_BTK_150.log'
fname = './eval_gtp2_1316_BTK_150.log'
f =  open(fname,'r')
lines = f.readlines()

predicts = []
truth = []
for line in lines:
    if 'startofsmiles' in line:
        # tmp = line.split('>')[1].strip()
        tmp = line.split('<|startofsmiles|>')
        # predicts.append(tmp[1].split(' ')[0].strip())
        predicts.append(tmp[-1].strip())

        # break
    elif 'Reference' in line:
        tmp = line.split('Reference smiles: ')[1].strip()
        truth.append(tmp)
    else:
        pass

nums = len(predicts)

In [15]:
import joblib
clf = joblib.load("SVC.m")

In [16]:
fname = f'../data/train/BTK.csv'
df_train = pd.read_csv(fname)
smiles_train_list = df_train['smiles'].values

In [ ]:
positive = []
negtive = []
for i in range(nums):
    smile = predicts[i]
    if smile in smiles_train_list:
        continue
    try:
        mol=Chem.MolFromSmiles(smile)
    except Exception as e:
        continue

    if mol is None:
        continue

    fp = MACCSkeys.GenMACCSKeys(mol)
    fp_bits = fp.ToList()
    label = clf.predict(np.array(fp_bits).reshape(1,-1)).item()
    if label == 1:
        positive.append(smile)
    elif label == 0:
        negtive.append(smile)

In [18]:
print(len(positive))
print(len(negtive))

1010
901


In [19]:
output_positive = []
for smile in positive:
    output_positive.append([smile, 1])

output_negtive = []
for smile in negtive:
    output_negtive.append([smile, 0])

output_positive = pd.DataFrame(data=output_positive, columns=['smiles', 'label'])
output_positive.to_csv(f'./data/positive.csv', index=False)

output_negtive = pd.DataFrame(data=output_negtive, columns=['smiles', 'label'])
output_negtive.to_csv(f'./data/negtive.csv', index=False)
print('Saved.')

Saved.
